In [128]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [129]:
df = pd.read_csv("Bank_Personal_Loan_Modelling.csv")
data.drop(["ZIP Code","ID","CD Account","Securities Account","Personal Loan"],axis=1,inplace=True)
y = df.pop('Credit Card')
x = df
x = StandardScaler().fit_transform(x)

In [130]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [131]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(tf.keras.Input(shape=(8,)))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [132]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# Define the PSO parameters
n_particles = 10
n_iterations = 50
c1 = 0.5
c2 = 0.5
w = 0.9
v_max = 0.5


In [ ]:
# Define the fitness function
def fitness_function(weights):
    # Split the flattened weights into weight matrices
    shapes = [w.shape for w in model.get_weights()]
    weights = np.split(weights, np.cumsum([np.prod(s) for s in shapes])[:-1])
    weights = [w.reshape(s) for w, s in zip(weights, shapes)]

    # Set the weights of the model
    model.set_weights(weights)

    # Evaluate the model
    loss, acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    return loss

In [ ]:
# Initialize the particles
positions = np.random.uniform(-1, 1, size=(n_particles, 417))
velocities = np.zeros_like(positions)

# Initialize the global best position and fitness
global_best_position = None
global_best_fitness = np.inf

In [ ]:
# Run the PSO algorithm
for i in range(n_iterations):
    # Evaluate the fitness of each particle
    fitness_values = np.array([fitness_function(p) for p in positions])
    
    # Update the global best position and fitness
    global_best_index = np.argmin(fitness_values)
    if fitness_values[global_best_index] < global_best_fitness:
        global_best_fitness = fitness_values[global_best_index]
        global_best_position = positions[global_best_index]
        
    # Update the velocities and positions of each particle
    for j in range(n_particles):
        r1 = np.random.uniform(0, 1)
        r2 = np.random.uniform(0, 1)
        
        local_best_position = positions[j]
        local_best_fitness = fitness_values[j]
        
        velocity = w * velocities[j] + c1 * r1 * (local_best_position - positions[j]) + c2 * r2 * (global_best_position - positions[j])
        velocity = np.clip(velocity, -v_max, v_max)
        
        positions[j] += velocity
        positions[j] = np.clip(positions[j], -1, 1)

In [ ]:
model.fit(x=x_train, y=y_train, verbose=1, epochs=50)

In [134]:
# Accuracy without using PSO
accuracy = model.evaluate(x=x_test, y=y_test, verbose=0)[1]
print(f"Test accuracy: {accuracy*100:.4f}%")

Test accuracy: 97.6800%


In [ ]:
# Accuracy using PSO
accuracy = model.evaluate(x=x_test, y=y_test, verbose=0)[1]
print(f"Test accuracy: {accuracy*100:.4f}%")